In [279]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [282]:
vod = pd.read_csv('../data/vod89.csv')
vod

,userid,program,main_cat,sub_cat,score,category,rename
0,59879000,소방서 옆 경찰서,TV드라마,기타,0.244476,TV프로그램,소방서옆경찰서
1,59879000,신성한 이혼,TV드라마,기타,0.738281,TV프로그램,신성한이혼
2,59895000,금이야 옥이야,TV드라마,기타,1.000000,TV프로그램,금이야옥이야
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,0.215482,TV프로그램,2022역사저널그날
4,59900000,그것이알고싶다,TV 시사/교양,기타,0.374741,TV프로그램,그것이알고싶다
...,...,...,...,...,...,...,...
1494,67140000,경남 통영 2부,우리동네,연예/오락,0.292893,TV프로그램,경남통영2부
1495,67140000,밀수,영화,액션/어드벤쳐,0.250000,영화,밀수
1496,67140000,잠자는 숲속의 공주,키즈,기타,0.292893,키즈,잠자는숲속의공주
1497,67148000,타요의 씽씽극장 동요2,키즈,기타,0.988951,키즈,타요의씽씽극장동요2


In [283]:
vod_id = pd.read_csv('../data/vod_add_id.csv', index_col=0)
vod_id = vod_id[['rename', 'Category', 'vod_id']]
vod_id.columns = ['rename', 'category', 'vod_id']
vod_id

,rename,category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
4710,베놈2:렛데어비카니지,영화,4710
4711,성월동화,영화,4711
4712,다큐인사이트코드블루,TV프로그램,4712
4713,캠프제로베이스원,TV프로그램,4713


In [284]:
vod1 = vod.merge(vod_id, on = ['rename', 'category'], how = 'left')
userid = pd.DataFrame(vod1['userid'].unique())
userid.columns = ['userid']
userid['user_id'] = userid.index

vod1 = vod1.merge(userid, on = ['userid'], how = 'left')
vod1

,userid,program,main_cat,sub_cat,score,category,rename,vod_id,user_id
0,59879000,소방서 옆 경찰서,TV드라마,기타,0.244476,TV프로그램,소방서옆경찰서,2092,0
1,59879000,신성한 이혼,TV드라마,기타,0.738281,TV프로그램,신성한이혼,1190,0
2,59895000,금이야 옥이야,TV드라마,기타,1.000000,TV프로그램,금이야옥이야,784,1
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,0.215482,TV프로그램,2022역사저널그날,572,2
4,59900000,그것이알고싶다,TV 시사/교양,기타,0.374741,TV프로그램,그것이알고싶다,297,2
...,...,...,...,...,...,...,...,...,...
1494,67140000,경남 통영 2부,우리동네,연예/오락,0.292893,TV프로그램,경남통영2부,4396,329
1495,67140000,밀수,영화,액션/어드벤쳐,0.250000,영화,밀수,2865,329
1496,67140000,잠자는 숲속의 공주,키즈,기타,0.292893,키즈,잠자는숲속의공주,689,329
1497,67148000,타요의 씽씽극장 동요2,키즈,기타,0.988951,키즈,타요의씽씽극장동요2,880,330


In [285]:
vod1.to_csv('../data/vod_score.csv', index = 0)

In [339]:
vod_score = pd.read_csv('../data/vod_score.csv')
vod_score = vod_score[['user_id', 'vod_id', 'score']]
vod_score

,user_id,vod_id,score
0,0,2092,0.244476
1,0,1190,0.738281
2,1,784,1.000000
3,2,572,0.215482
4,2,297,0.374741
...,...,...,...
1494,329,4396,0.292893
1495,329,2865,0.250000
1496,329,689,0.292893
1497,330,880,0.988951


In [340]:
# vod_score['score'] = pd.cut(vod_score['score'], bins = [0,0.2,0.4,0.6,0.8, 1], labels = [1,2,3,4,5])
# vod_score

In [341]:
from sklearn.utils import shuffle
TRAIN_SIZE = 0.8
vod_score = shuffle(vod_score)
cutoff = int(TRAIN_SIZE * len(vod_score))

vod_train = vod_score.iloc[:cutoff]
vod_test = vod_score.iloc[cutoff:]

In [342]:
# from sklearn.model_selection import train_test_split

# vod_train, vod_test = train_test_split(vod_score, test_size = 0.2, stratify=vod_score['user_id'], random_state = 42)
# vod_train

In [343]:
import numpy as np
import pandas as pd

# 필요한 Surprise 알고리즘 불러오기
from surprise import BaselineOnly, KNNWithMeans, SVD, SVDpp, NMF, Dataset, accuracy, Reader, dump
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(0, 5))
    trainset = Dataset.load_from_df(training_dataframe[['user_id', 'vod_id', 'score']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['user_id', 'vod_id', 'score']], reader)

    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    
    return trainset, testset

trainset, testset = convert_traintest_dataframe_forsurprise(vod_train, vod_test)

In [344]:
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from surprise import dump
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

sim_options = {'name': 'cosine',
               'user_based': True  # user-based similarity
               }
knnbaseline = KNNBaseline(sim_options=sim_options)

knnbaseline.fit(trainset)
knnbaseline_predictions = knnbaseline.test(testset)

# file_name = 'KnnBaseline_model_u'
# dump.dump(file_name, algo=knnbaseline_predictions)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(knnbaseline_predictions)
accuracy.mae(knnbaseline_predictions)
print("Done!")


Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.2296
MAE:  0.1584
Done!


In [345]:
user_id = sorted(vod_score.user_id.unique())
vod_id = sorted(vod_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['user_id', 'vod_id', 'real', 'predict'])
result = result[['user_id', 'vod_id', 'predict']]
result

,user_id,vod_id,predict
0,0,0,0.397929
1,0,3,0.425983
2,0,6,0.284225
3,0,7,0.364404
4,0,8,0.456492
...,...,...,...
270907,331,4632,0.395008
270908,331,4645,0.397470
270909,331,4683,0.397470
270910,331,4696,0.391499


In [346]:
result_1 = []
for user in user_id:
    result_1.append(result[result['user_id'] == user].sort_values(by = 'predict', ascending=False).vod_id[0:15].values.tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['user_id']), result_1], axis = 1)
result_1

,user_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,1156,1190,516,247,880,2092,533,1166,296,143,818,3927,888,401,1232
1,1,14,738,1277,1156,880,1885,784,1166,842,143,320,818,888,1232,336
2,2,858,720,315,824,112,123,739,2092,1166,15,356,754,1077,450,530
3,3,754,356,1077,114,706,1312,836,124,398,323,751,888,886,880,1166
4,4,796,728,1069,955,36,581,880,1166,296,143,1156,818,888,401,1232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,327,818,880,1166,296,143,1156,888,401,1232,336,1233,848,335,8,747
328,328,780,880,1166,296,143,47,1156,818,888,401,1232,336,1233,848,335
329,329,991,726,2226,862,1166,880,296,143,1515,1686,1156,818,888,401,1232
330,330,830,880,1166,296,143,1156,818,888,401,1232,336,1233,848,335,8


In [347]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['user_id', 'vod_id']
vod_predict

,user_id,vod_id
0,0,"[1156, 1190, 516, 247, 880, 2092, 533, 1166, 2..."
1,1,"[14, 738, 1277, 1156, 880, 1885, 784, 1166, 84..."
2,2,"[858, 720, 315, 824, 112, 123, 739, 2092, 1166..."
3,3,"[754, 356, 1077, 114, 706, 1312, 836, 124, 398..."
4,4,"[796, 728, 1069, 955, 36, 581, 880, 1166, 296,..."
...,...,...
327,327,"[818, 880, 1166, 296, 143, 1156, 888, 401, 123..."
328,328,"[780, 880, 1166, 296, 143, 47, 1156, 818, 888,..."
329,329,"[991, 726, 2226, 862, 1166, 880, 296, 143, 151..."
330,330,"[830, 880, 1166, 296, 143, 1156, 818, 888, 401..."


In [348]:
vod_test

,user_id,vod_id,score
970,211,3847,0.292893
888,182,1430,0.292893
957,206,7,0.292893
1417,309,778,0.714282
1104,253,2288,0.292893
...,...,...,...
484,96,2312,0.292893
878,176,335,0.157677
1015,228,1520,0.323223
1469,319,24,0.292893


In [349]:
testdata = vod_test.groupby('user_id')['vod_id'].unique().reset_index()
# index = testdata.vod_id.apply(lambda x : len(x)).values > 0
# testdata = testdata[index].reset_index(drop = True)
testdata

,user_id,vod_id
0,2,[3314]
1,4,"[1190, 456, 2846]"
2,8,"[887, 3946, 381, 3738]"
3,12,"[1166, 2712, 747]"
4,15,"[1398, 3185]"
...,...,...
139,319,[24]
140,321,"[111, 341, 289, 2537]"
141,324,[2537]
142,326,[3850]


In [350]:
# precision@15
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='user_id', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2]))) / K
        precision_value += temp
        
    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict)

0.010416666666666668